Project 1 - Baseball Analysis

In [13]:
#Import Dependencies
import pandas as pd

In [14]:
#Read in Teams CSV Data as DataFrame
teams_filepath = "Resources/Teams.csv"
teams_df = pd.read_csv(teams_filepath)

In [15]:
#Organize Teams Dataframe Columns (Put Columns in Order, Get Rid of Columns you dont need)
teams_df = teams_df[["name", "yearID", "G", "W", "L", "R", "AB", "H", "2B", "3B", "HR", "BB", "SO", "SB", "ER", "ERA", "CG", "E"]]


In [16]:
#Drop all years prior to 1960 (Just analyzing the modern era of baseball ~1960)
teams_df = teams_df.loc[teams_df["yearID"] > 1959]
teams_df.head()

,name,yearID,G,W,L,R,AB,H,2B,3B,HR,BB,SO,SB,ER,ERA,CG,E
1343,Baltimore Orioles,1960,154,89,65,682,5170,1307,206,33,123,596,801.0,37.0,538,3.52,48,107
1344,Boston Red Sox,1960,154,65,89,658,5215,1359,234,32,124,570,798.0,34.0,699,4.62,34,140
1345,Chicago White Sox,1960,154,87,67,741,5191,1402,242,38,112,567,648.0,122.0,552,3.60,42,109
1346,Chicago Cubs,1960,156,60,94,634,5311,1293,213,48,119,531,897.0,51.0,678,4.35,36,143
1347,Cincinnati Reds,1960,154,67,87,640,5289,1324,230,40,140,512,858.0,73.0,618,4.00,33,125


In [17]:
#Read in Salary CSV Data as DataFrame
salary_csvpath = "Resources/Salaries.csv"
salary_df = pd.read_csv(salary_csvpath)
salary_df.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [18]:
#Need to Retrieve Team Total Salary Caps by Year from Salary Dataframe